### Content based Recommender Systems

##### Dataset : https://www.kaggle.com/datafiniti/consumer-reviews-of-amazon-products

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error

In [75]:
import csv
df = pd.read_csv(r'/home/exoper/Downloads/datafiniti-consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')


In [76]:
print(df.columns)
print(df.shape)



Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')
(5000, 24)


In [77]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [78]:
count = df.groupby("asins", as_index=False).count()
mean = df.groupby("asins", as_index=False).mean()

dfMerged = pd.merge(df, count, how='right', on=['asins'])
dfMerged

,id_x,dateAdded_x,dateUpdated_x,name_x,asins,brand_x,categories_x,primaryCategories_x,imageURLs_x,keys_x,...,reviews.dateSeen_y,reviews.doRecommend_y,reviews.id_y,reviews.numHelpful_y,reviews.rating_y,reviews.sourceURLs_y,reviews.text_y,reviews.title_y,reviews.username_y,sourceURLs_y
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
5,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
6,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
7,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
8,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96
9,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,96,96,5,96,96,96,96,96,96,96


In [79]:
df1 = df[['reviews.text','reviews.rating','asins']]
df1 = df1.dropna()
df1

,reviews.text,reviews.rating,asins
0,I thought it would be as big as small paper bu...,3,B00ZV9PXP2
1,This kindle is light and easy to use especiall...,5,B00ZV9PXP2
2,Didnt know how much i'd use a kindle so went f...,4,B00ZV9PXP2
3,I am 100 happy with my purchase. I caught it o...,5,B00ZV9PXP2
4,Solid entry level Kindle. Great for kids. Gift...,5,B00ZV9PXP2
5,This make an excellent ebook reader. Don't exp...,5,B00ZV9PXP2
6,"I ordered this for my daughter, as I have the ...",5,B00ZV9PXP2
7,I bought my Kindle about 2 months ago and the ...,4,B00ZV9PXP2
8,"amazon kindle is always the best ebook, upgrad...",5,B00ZV9PXP2
9,"It's beyond my expectation, and it can even sh...",5,B00ZV9PXP2


In [80]:
df1.asins.unique()

array(['B00ZV9PXP2', 'B010CEHQTG', 'B01J24C0TI', 'B01N32NCPM',
       'B06XB29FPF', 'B00QFQRELG', 'B018Y22C2Y', 'B017JG41PC',
       'B018Y22BI4', 'B018Y23MNM', 'B01AHBBG04', 'B01AHBDCKQ',
       'B01BH83OOM', 'B0189XYY0Q', 'B00VINDBJK', 'B00IOY8XWQ',
       'B01AHB9CN2', 'B018Y225IA', 'B00IOYAM4I', 'B01AHB9C1E',
       'B00REQKWGA', 'B01ACEKAJY', 'B01AHB9CYG', 'B018Y224PY'],
      dtype=object)

In [81]:
dfProductReview = df.groupby("asins", as_index=False).mean()
dfProductReview.head(3)

,asins,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating
0,B00IOY8XWQ,0.951220,NaN,1.512195,4.646341
1,B00IOYAM4I,1.000000,NaN,4.090909,4.818182
2,B00QFQRELG,0.954545,NaN,0.181818,4.863636


### Gropuing reviews of individual products

In [82]:
ProductReviewSummary = df1.groupby("asins")["reviews.text"].apply(str)

In [83]:
p = ProductReviewSummary.to_frame()
p['reviews.text'] = p['reviews.text'].str.replace('\d+'," ")
p['reviews.text'] = p['reviews.text'].str.replace('\n'," ")
p['reviews.text'] = p['reviews.text'].str.strip(" ")


In [84]:
p.shape[0]

24

In [85]:
# import nltk
# from nltk import sent_tokenize,word_tokenize,pos_tag
# from nltk.corpus import stopwords
# text = "Machine learning is the science of getting computers to act without being explicitly programmed. In the past decade, machine learning has given us self-driving cars, practical speech recognition, effective web search, and a vastly improved understanding of the human genome. Machine learning is so pervasive today that you probably use it dozens of times a day without knowing it. Many researchers also think it is the best way to make progress towards human-level AI. In this class, you will learn about the most effective machine learning techniques, and gain practice implementing them and getting them to work for yourself. More importantly, you'll learn about not only the theoretical underpinnings of learning, but also gain the practical know-how needed to quickly and powerfully apply these techniques to new problems. Finally, you'll learn about some of Silicon Valley's best practices in innovation as it pertains to machine learning and AI."
# t = text.replace(','," ")


In [655]:
# stp = set(stopwords.words('english'))
# sym1 = [',','.']
# sen = sent_tokenize(t)
# sents = word_tokenize(t)
# tagged_tokens  = pos_tag(sents)
# wordlist = [w for w in sents if not w in stp ]
# w =[word for word in wordlist if word != '.']

# print(sen)

In [656]:
# train = []
# for ix in sen[:]:
#     train.append(ix.lower().split())
# train

In [657]:
# import gensim

In [658]:
# model = gensim.models.Word2Vec(train, workers=2 , iter=20, )

In [193]:
# model.similarity('machine','learning')

/home/exoper/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/home/exoper/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.13365737

## Tfidf matrix and cosine similarity

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [87]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [88]:
tfidf_matrix = tf.fit_transform(p['reviews.text'])


In [89]:
print((tfidf_matrix.shape))

(24, 9968)


In [90]:
cosine_similarities = cosine_similarity(tfidf_matrix,Y=None,dense_output=False)
cnum = (cosine_similarities.toarray())
print(((cosine_similarities[0][:1,:-19])))
#print(cnum[1])
type(cosine_similarities)
#sorted(cosine_similarities[0] , reverse = True)

  (0, 2)	0.0724764429125068
  (0, 1)	0.11608388967434019
  (0, 4)	0.15367614643805444
  (0, 3)	0.12171484598360283
  (0, 0)	1.0000000000000002


scipy.sparse.csr.csr_matrix

## Recommendations

In [93]:
def get_recommendations(id):
    print("the product selected is {}".format(p.index[id]))

    a = cosine_similarities.getcol(id)
    val = list(enumerate(a.data))
    #print(val)
    b= dict(val)
    print(b)
    c = sorted(b.items(),key=lambda x:x[1],reverse=True)[1:4]
    k = 1
    for idx in c:
        print("The {} Recommendation is {}".format(k,p.index[idx[0]]))
        k += 1

In [94]:
get_recommendations(0)

the product selected is B00IOY8XWQ
{0: 1.0000000000000002, 1: 0.11608388967434019, 2: 0.0724764429125068, 3: 0.12171484598360283, 4: 0.15367614643805444, 5: 0.1617595148834279, 6: 0.04779646032122106, 7: 0.13546786196538704, 8: 0.10171803450269766, 9: 0.07022880070980797, 10: 0.09366838925255032, 11: 0.07615076798068991, 12: 0.0663242482797254, 13: 0.04079763253226438, 14: 0.11789250758191805, 15: 0.10168905975552082, 16: 0.10852428478472535, 17: 0.07716139838348668, 18: 0.07904046896964702, 19: 0.11599822981570619, 20: 0.04014152022720062, 21: 0.045100144993007366, 22: 0.019073090955666207, 23: 0.0437746295389118}
The 1 Recommendation is B00ZV9PXP2
The 2 Recommendation is B00VINDBJK
The 3 Recommendation is B017JG41PC


## CountVector and knn

In [99]:
reviews = p['reviews.text']
countVector = CountVectorizer(max_features = 300, stop_words='english') 
transformedReviews = countVector.fit_transform(reviews) 

dfReviews = pd.DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)
type(transformedReviews)

scipy.sparse.csr.csr_matrix

In [100]:
dfReviews.index


RangeIndex(start=0, stop=24, step=1)

In [54]:
dfReviews1 = pd.DataFrame(tfidf_matrix.A, columns = tf.get_feature_names())
dfReviews1 = dfReviews1.astype(int)

In [55]:
dfReviews1.head(3)

,ab,ab bought,ab bought children,ab enjoy,ab enjoy summer,ab gift,ab gift fot,ab great,ab great way,ability,...,äôs lot fun,äôt,äôt used,äôt used gift,äôve,äôve enjoy,äôve enjoy echo,äústar,äústar trek,äústar trek ti
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
X = np.array(dfReviews)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]
#len of train and test
lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)
X.shape

(24, 300)

In [70]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(dfReviews_train)

distances,indices = neighbor.kneighbors(dfReviews_train)
print(distances,indices)

[[ 0.         21.49418526 21.51743479]
 [ 0.         13.45362405 13.52774926]
 [ 0.         13.45362405 14.28285686]
 [ 0.         12.489996   17.91647287]
 [ 0.         12.489996   13.52774926]
 [ 0.         15.32970972 21.49418526]
 [ 0.         29.27456234 30.        ]
 [ 0.         15.32970972 21.86321111]
 [ 0.         21.1896201  21.84032967]
 [ 0.         15.77973384 18.81488772]
 [ 0.         15.77973384 19.13112647]
 [ 0.         18.52025918 18.81488772]
 [ 0.         18.54723699 24.35159132]
 [ 0.         18.54723699 24.87971061]
 [ 0.         16.673332   17.05872211]
 [ 0.         17.05872211 17.20465053]
 [ 0.         19.13112647 20.32240143]
 [ 0.         16.0623784  16.70329309]
 [ 0.         19.31320792 21.51743479]
 [ 0.         16.673332   18.24828759]
 [ 0.         25.67099531 26.77685568]] [[ 0  5  4]
 [ 1  2  4]
 [ 2  1  4]
 [ 3  4  1]
 [ 4  3  1]
 [ 5  7  0]
 [ 6 20  1]
 [ 7  5  4]
 [ 8 15 17]
 [ 9 10 11]
 [10  9 16]
 [11 15  9]
 [12 13 11]
 [13 12  9]
 [14 19 15]


In [111]:
for i in range(lentest):
    a = neighbor.kneighbors([dfReviews_test[i]])
    related_product_list = a[1]
    #print(related_product_list)
#     first_related_product = [item[0] for item in related_product_list]
#     first_related_product = str(first_related_product).strip('[]')
#     first_related_product = int(first_related_product)
#     print(first_related_product)

#     second_related_product = [item[1] for item in related_product_list]
#     second_related_product = str(second_related_product).strip('[]')
#     second_related_product = int(second_related_product)
#     print(second_related_product)
    print(lentrain+i)
    print ("Based on product reviews, for ", p.index[lentrain+i])
    print ("The first similar product is ", p.index[first_related_product])
    print ("The second similar product is ", p.index[second_related_product]) #
    print('------------------------------------------------------------------')


21
Based on product reviews, for  B01J24C0TI
The first similar product is  B01BH83OOM
The second similar product is  B010CEHQTG
------------------------------------------------------------------
22
Based on product reviews, for  B01N32NCPM
The first similar product is  B01BH83OOM
The second similar product is  B010CEHQTG
------------------------------------------------------------------
23
Based on product reviews, for  B06XB29FPF
The first similar product is  B01BH83OOM
The second similar product is  B010CEHQTG
------------------------------------------------------------------


In [127]:
X = neighbor.kneighbors([dfReviews_test[1]])
X

(array([[11.61895004, 12.56980509, 15.29705854]]), array([[1, 2, 4]]))

In [128]:
dfReviews_test[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], d